In [1]:
import pandas as pd
import numpy as np

In [ ]:
log_g_preds = pd.read_csv("test_preds/log_preds_g.csv")
log_f_preds = pd.read_csv("test_preds/log_preds_f.csv")
log_c_preds = pd.read_csv("test_preds/log_preds_c.csv")



In [ ]:
rf_g_preds = pd.read_csv("test_preds/rf_preds_g.csv")
rf_f_preds = pd.read_csv("test_preds/rf_preds_f.csv")
rf_c_preds = pd.read_csv("test_preds/rf_preds_c.csv")

xg_g_preds = pd.read_csv("test_preds/xg_preds_g.csv")
xg_f_preds = pd.read_csv("test_preds/xg_preds_f.csv")
xg_c_preds = pd.read_csv("test_preds/xg_preds_c.csv")